# bert test

This is a text cell. Start editing!

In [2]:
import com.johnsnowlabs.nlp.SparkNLP
import com.johnsnowlabs.nlp.annotator._
import com.johnsnowlabs.nlp.base._
import org.apache.spark.ml.{Pipeline, PipelineModel, Transformer}
import org.apache.spark.sql.types._
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions.{udf,to_timestamp}
import org.apache.spark.storage._
import org.apache.spark.ml.feature._
import org.apache.spark.ml.classification._
import org.apache.spark.ml.linalg.DenseVector
import org.apache.spark.ml.param.{Param, ParamMap}
import org.apache.spark.ml.util.{DefaultParamsReadable, DefaultParamsWritable, Identifiable}
import org.apache.spark.sql.{DataFrame, Dataset}
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.{col, explode, udf}
import org.apache.spark.sql.types.{DataTypes, StructType}
import scala.collection.mutable.WrappedArray



In [1]:
val bert = BertEmbeddings.pretrained(name="bert_multi_cased", lang="xx")

bert_multi_cased download started this may take some time.
Approximate size to download 638.7 MB
Download done! Loading the resource.


In [6]:
val sentenceStartTokenId = 101
val sentenceEndTokenId = 102

In [5]:
val docs = Seq(Array(123,119, 11025), Array(10135))
val prep = docs.map(tokens => {
    Array(sentenceStartTokenId) ++
      tokens ++
      Array(sentenceEndTokenId) ++
      Array.fill(bert.getMaxSentenceLength - tokens.length - 2)(0)
})
prep

List([I@4d56e7bd, [I@be3b8ba)

In [8]:
val tfBert = bert.getModelIfNotSet
tfBert

com.johnsnowlabs.ml.tensorflow.TensorflowBert@51f6faa2

In [4]:
// Code to use embeddings of bert
tfBert.tag(prep, "module/bert/encoder/Reshape_1:0", bert.getMaxSentenceLength)

List([[F@5d54059d, [[F@4104bc2)

In [3]:
val trainDf = Seq(0 to 100:_*).toDF()
    .withColumn("tokens", when(rand() > 0.2, Array(101, 123,119, 11025, 102)).otherwise(Array(101, 10135, 102)))

In [7]:
// val average_embeddings = 
val udf_aggBert = udf[Array[Float], WrappedArray[Int]](tokens => {
            val input = Seq(tokens.toArray ++ Array.fill(bert.getMaxSentenceLength - tokens.length)(0))
            val embeddings = tfBert.tag(input, "module/bert/encoder/Reshape_1:0", bert.getMaxSentenceLength)
            val sumEmbeddings = embeddings.head.transpose.map(_.sum)
            sumEmbeddings.map(x => (x * 1.0 / bert.getMaxSentenceLength).toFloat)
})
trainDf.withColumn("embeddings", udf_aggBert('tokens)).show

+-----+--------------------+--------------------+
|value|              tokens|          embeddings|
+-----+--------------------+--------------------+
|    0|[101, 123, 119, 1...|[0.69951665, 0.43...|
|    1|[101, 123, 119, 1...|[0.69951665, 0.43...|
|    2|[101, 123, 119, 1...|[0.69951665, 0.43...|
|    3|   [101, 10135, 102]|[0.71374446, 0.42...|
|    4|[101, 123, 119, 1...|[0.69951665, 0.43...|
|    5|[101, 123, 119, 1...|[0.69951665, 0.43...|
|    6|[101, 123, 119, 1...|[0.69951665, 0.43...|
|    7|[101, 123, 119, 1...|[0.69951665, 0.43...|
|    8|[101, 123, 119, 1...|[0.69951665, 0.43...|
|    9|[101, 123, 119, 1...|[0.69951665, 0.43...|
|   10|[101, 123, 119, 1...|[0.69951665, 0.43...|
|   11|   [101, 10135, 102]|[0.71374446, 0.42...|
|   12|[101, 123, 119, 1...|[0.69951665, 0.43...|
|   13|[101, 123, 119, 1...|[0.69951665, 0.43...|
|   14|[101, 123, 119, 1...|[0.69951665, 0.43...|
|   15|[101, 123, 119, 1...|[0.69951665, 0.43...|
|   16|[101, 123, 119, 1...|[0.69951665, 0.43...|
